In [0]:
import os
import sys
import pandas as pd
import numpy as np
import scipy as sp
import scandir
import glob
import shutil
from ipyparallel import Client

In [0]:
ddocent = "/home/cfriedline/g/src/dDocent_run/dDocent"

In [0]:
fastq_dirs = ["/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/I1", "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/I3"]

In [0]:
fastq_files = []
for d in fastq_dirs:
    fs = [os.path.join(d, x) for x in scandir.listdir(d) if "fastq" in x and not "lane" in x]
    for elem in fs:
        fastq_files.append(elem)

In [0]:
fastq_files

In [0]:
trans = pd.read_csv("translation_table.csv", sep="\t")
trans.index = trans.illumina_id
trans = trans.drop("illumina_id", axis=1)
trans['name'] = trans.apply(lambda x: "{}_{}".format(x['pop'], x.indiv), axis=1)
trans_dict = trans.T.to_dict()

In [0]:
analysis_dir = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/dDocent/"

In [0]:
rc = Client(profile="sge")

In [0]:
dv = rc[:]
lv = rc.load_balanced_view()
len(dv)

In [0]:
with dv.sync_imports():
    import os
    import shutil

In [0]:
def copy_file(args):
    src, dst = args
    shutil.copy(src, dst)
    return dst

In [0]:
dv['copy_file'] = copy_file

In [0]:
jobs = []
for i, f in enumerate(fastq_files):
    fname = os.path.basename(f).split(".")[0]
    if not fname in trans_dict:
        print(fname)
    dst = os.path.join(analysis_dir, "{}.F.fq".format(trans_dict[fname]['name']))
    if not "OTIS" in fname:
        jobs.append(lv.apply_async(copy_file, (f, dst)))

In [0]:
np.sum([x.ready() for x in jobs]), len(jobs)

### Compress fastq files

```
cd /home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/dDocent
ls *.fq | parallel bgzip {}
```

### Choices during dDocent

* Number of Unique Sequences with More than X Coverage (Counted within individuals): 3
* Number of Unique Sequences present in more than X Individuals: 17 (10%)

### makefile

```
>cat /home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/dDocent/makefile

default:
    export PATH=~/g/src/dDocent_run:~/g/src/dDocent_run/bwa:~/g/src/dDocent_run/seqtk:~/g/src/vcftools_0.1.11/bin:$$PATH; ~/g/src/dDocent_run/dDocent
```

### Run

```
qrsh -N dDocent -pe smp 40 -q godel199@godel96
cd /home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/dDocent
make
```
